#1. 사이킷런 KNN 으로 MNIST 분류하기, 좋은 파라미터를 찾는 방법을 알아보자.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 데이터셋 불러오기, 
mnist = fetch_openml('mnist_784', as_frame=False)
X, y = mnist["data"], mnist["target"]
print(X.shape, y.shape) # 자료 크기

# 교육용 5만개 분리
num = 50000
X_train, X_test, y_train, y_test = X[:num], X[num:], y[:num], y[num:]

# 트레인 세트 셔플, 
shuffle_index = np.random.permutation(num)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# knn으로 피팅
knn_clf = KNeighborsClassifier(n_jobs=-1,weights ='distance', n_neighbors=3)
knn_clf.fit(X_train,y_train)

# knn 피팅한 모델과 테스트셋 정밀도 
y_knn_pred = knn_clf.predict(X_test)
accuracy_score(y_test,y_knn_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_knn_pred))

In [ ]:
knn_clf2 = KNeighborsClassifier(n_jobs=-1,weights ='distance', n_neighbors=4)
knn_clf2.fit(X_train,y_train)

# knn 피팅한 모델과 테스트셋 정밀도 
y_knn_pred2 = knn_clf2.predict(X_test)
accuracy_score(y_test,y_knn_pred2)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_knn_pred2))

#2. MNIST 데이터를 MLPClassifier 만들기

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_mlp_pred = mlp.predict(X_test)
accuracy_score(y_test,y_mlp_pred)

In [ ]:
mlp

In [ ]:
from time import time
i = 0
df = pd.DataFrame(columns = ['alpha','max_iter','train_acc','test_acc','train_time'])
for a in [0.00001,0.0001,0.001,0.01, 0.1, 1, 10]:
    for mi in [10,100,200,500,1000,2000]:
        st = time()
        mlp = MLPClassifier(alpha=a, max_iter=mi)
        mlp.fit(X_train, y_train)
        end = time() - st
        
        acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
        acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
        df.loc[i] = [a,mi,acc_tr,acc,end]
        i=i+1

In [ ]:
df

#3. mnist 데이터 GaussianNB 만들기

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb

In [ ]:
gnb.fit(X_train,y_train)

# gnb 피팅한 모델과 테스트셋 정밀도 
y_gnb_pred = gnb.predict(X_test)
accuracy_score(y_test,y_gnb_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_gnb_pred))

In [ ]:
print(confusion_matrix(y_train,gnb.predict(X_train)))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(gnb, X_train, y_train, cv=3, scoring="accuracy")

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb

In [ ]:
mnb.fit(X_train,y_train)

# gnb 피팅한 모델과 테스트셋 정밀도 
y_mnb_pred = mnb.predict(X_test)
accuracy_score(y_test,y_mnb_pred)

#4. 아이리스 데이터셋 꽃잎길이로 setosa를 분류

In [ ]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

iris = datasets.load_iris()

# setosa 만 True
print(iris['target_names']) # setosa = 0번 자료
y = (iris["target"] == 0).astype(np.int)

print(iris['feature_names']) # setosa = 0번 자료
X = iris["data"][:,2].reshape(-1,1) # reshape 안맞춰주면 에러남

test = [[5.7],[1.5]]
log_reg = LogisticRegression().fit(X, y)
predic = log_reg.predict_proba(test)

for i in range(len(test)):
    print("\n꽃잎의 길이 :",*test[i])
    if predic[i][0] >= 0.5 : print(int(100*predic[i][0]),"% 확률로 setosa")
    else : print(int(100*predic[i][1]),"% 확률로 setosa 아님")

In [ ]:
# 그림 그릴 자료

X_new = np.linspace(0,10,1000).reshape(-1,1)
y_proba = log_reg.predict_proba(X_new) # 확률을 나타냄
decision_boundary = X_new[y_proba[:,1]<=0.5][0] 
# 확률이 50%가 넘어가는 X_new 중에 가장 낮은  요소

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,3))
plt.plot(X[y==0],y[y==0],"bs")
plt.plot(X[y==1],y[y==1],"g^")

plt.plot(X_new,y_proba[:,1],"g-",linewidth=2, label="iris-Setosa")
plt.plot(X_new,y_proba[:,0], "b--", linewidth=2, label="NOt iris-setosa")

plt.text(decision_boundary,0.6,"decision boundary",fontsize=14, color="k", ha="center")
plt.plot([decision_boundary, decision_boundary],[0,1],"k:",linewidth=2, label="decision boundary")
plt.arrow(decision_boundary, 0.8,0.3,0,head_width = 0.05, head_length=0.1, fc='b', ec='b')
plt.arrow(decision_boundary, 0.8,-0.3,0,head_width = 0.05, head_length=0.1, fc='g', ec='g')

plt.xlabel("length of sepal (cm)",fontsize=14)
plt.ylabel("Prabability",fontsize=14)
plt.legend(loc = "center right",fontsize=14)

plt.axis([0,10,-0.02,1.02])
plt.show()

#5. 아이리스 데이터 셋 테스트셋, 트레인셋 나누고 LDA, 소프트맥스 분류기 만들고 classification report와 confusion matrix 출력

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn import metrics 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# 데이터 불러오기
iris = datasets.load_iris()
X,y = iris['data'], iris['target']

# 8:2 로 데이터셋 나눔.
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=103, test_size=0.2, stratify=y)

softmax_reg = LogisticRegression(multi_class="multinomial",C=10,random_state=42, max_iter=1000, solver='lbfgs')
softmax_reg.fit(X_train,y_train)


In [ ]:
predicted_y = softmax_reg.predict(X_test)

print();print(metrics.classification_report(y_test,predicted_y))
print();print(metrics.confusion_matrix(y_test,predicted_y))

In [ ]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train,y_train)

expected_y = y_test
predicted_y = clf.predict(X_test)

print();print(metrics.classification_report(expected_y,predicted_y))
print();print(metrics.confusion_matrix(expected_y,predicted_y))